In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from pandas.tseries.offsets import DateOffset
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

In [2]:
df_sales = pd.read_csv('Sales_Data05.csv')
df_member = pd.read_csv('Member_Data.csv',encoding='cp949')
df_product = pd.read_csv('Product_Data.csv')

In [3]:
# 1. 데이터 전처리 및 고객별 피처 엔지니어링
# ================================================================

# (A) df_sales: 주문일시를 datetime으로 변환
df_sales['주문일시'] = pd.to_datetime(df_sales['주문일시'], errors='coerce')

# (B) 배송지연 여부 생성: (이미 생성되어 있지 않은 경우)
#     배송완료일을 datetime으로 변환 후, 배송기간(일수) 계산
df_sales['배송완료일'] = pd.to_datetime(df_sales['배송완료일'], errors='coerce')
df_sales['배송기간'] = (df_sales['배송완료일'] - df_sales['주문일시']).dt.days
threshold_days = 2  # 기준값, 이후 쉽게 조정 가능
df_sales['배송지연여부'] = (df_sales['배송기간'] > threshold_days).astype(int)

# (C) 고객별 구매 내역 집계: 총 주문 건수, 총 구매 금액, 배송지연 주문 수 계산
customer_orders = df_sales.groupby('회원번호').agg(
    total_orders = ('회원번호', 'count'),
    total_purchase_amount = ('구매금액', 'sum'),
    shipping_delay_count = ('배송지연여부', 'sum')
).reset_index()
customer_orders['shipping_delay_freq'] = customer_orders['shipping_delay_count'] / customer_orders['total_orders']

# (D) 고객별 평균 구매 주기 계산: 주문일시 간 차이를 일 단위로 계산
def compute_avg_cycle(dates):
    dates = dates.sort_values()
    if len(dates) < 2:
        return np.nan
    diffs = dates.diff().dropna().dt.days
    return diffs.mean()

purchase_cycle = df_sales.groupby('회원번호')['주문일시'].apply(compute_avg_cycle).reset_index(name='avg_purchase_cycle')

# (E) 고객별 마지막 주문일 계산
last_order = df_sales.groupby('회원번호')['주문일시'].max().reset_index(name='last_order_date')

# (F) 고객별 피처 병합
customer_features = customer_orders.merge(purchase_cycle, on='회원번호', how='left')
customer_features = customer_features.merge(last_order, on='회원번호', how='left')

# (G) 구독 여부 전처리: df_member의 '구독여부' 컬럼 전처리 (문자열인 경우)
df_member['구독여부'] = df_member['구독여부'].astype(str).str.strip().str.lower()
mapping_dict = {'true': True, 'false': False, 'yes': True, 'no': False}
df_member['구독여부'] = df_member['구독여부'].map(mapping_dict)

# (H) 고객 피처에 회원 정보(구독여부) 병합
customer_features = customer_features.merge(df_member[['회원번호', '구독여부']], on='회원번호', how='left')

# (I) 결측치 처리: avg_purchase_cycle NaN은 중앙값으로 채움
customer_features['avg_purchase_cycle'] = customer_features['avg_purchase_cycle'].fillna(customer_features['avg_purchase_cycle'].median())


In [10]:
# 2. 라벨 생성: 목표 변수 설정
# ================================================================

# 모델 1: 구독 확률 예측 -> target: 구독여부 (0/1)
# (이미 customer_features에 구독여부가 포함되어 있음)
# NaN 처리: 구독여부에 NaN이 있다면 0으로 처리 (미구독으로 가정)
customer_features['구독여부'] = customer_features['구독여부'].fillna(False).astype(int)

# 모델 2: 구매주기 n개월 이하(자주 구매) 예측
# 기준: n개월을 일(day)로 환산. (예: n=3개월이면 약 90일)
n_months = 1
threshold_days = n_months * 30  # 약 90일
customer_features['frequent_buyer'] = (customer_features['avg_purchase_cycle'] <= threshold_days).astype(int)


In [11]:
# 3. 모델 입력 변수 구성 및 실험
# ================================================================

# 선택할 피처 (입력 변수): 구매 내역, 구매 주기, 배송지연 빈도
feature_cols = ['total_orders', 'total_purchase_amount', 'avg_purchase_cycle', 'shipping_delay_freq']

# -----------------------
# 모델 1: 구독 확률 예측
# -----------------------
X_sub = customer_features[feature_cols]
y_sub = customer_features['구독여부']

X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(X_sub, y_sub, test_size=0.3, random_state=42)

log_reg_sub = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
log_reg_sub.fit(X_train_sub, y_train_sub)

y_pred_proba_sub = log_reg_sub.predict_proba(X_test_sub)[:, 1]
print("----- 구독 확률 예측 모델 -----")
print("ROC AUC Score (구독 모델):", roc_auc_score(y_test_sub, y_pred_proba_sub))
print(classification_report(y_test_sub, log_reg_sub.predict(X_test_sub)))

# -----------------------
# 모델 2: 자주 구매(구매주기 n개월 이하) 예측
# -----------------------
X_freq = customer_features[feature_cols]
y_freq = customer_features['frequent_buyer']

X_train_freq, X_test_freq, y_train_freq, y_test_freq = train_test_split(X_freq, y_freq, test_size=0.3, random_state=42)

log_reg_freq = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
log_reg_freq.fit(X_train_freq, y_train_freq)

y_pred_proba_freq = log_reg_freq.predict_proba(X_test_freq)[:, 1]
print("\n----- 자주 구매 (구매주기 n개월 이하) 예측 모델 -----")
print("ROC AUC Score (자주 구매 모델):", roc_auc_score(y_test_freq, y_pred_proba_freq))
print(classification_report(y_test_freq, log_reg_freq.predict(X_test_freq)))

----- 구독 확률 예측 모델 -----
ROC AUC Score (구독 모델): 0.5121570425623205
              precision    recall  f1-score   support

           0       0.89      0.22      0.35      3238
           1       0.15      0.83      0.25       524

    accuracy                           0.30      3762
   macro avg       0.52      0.52      0.30      3762
weighted avg       0.78      0.30      0.34      3762


----- 자주 구매 (구매주기 n개월 이하) 예측 모델 -----
ROC AUC Score (자주 구매 모델): 1.0
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       282
           1       1.00      1.00      1.00      3480

    accuracy                           1.00      3762
   macro avg       1.00      1.00      1.00      3762
weighted avg       1.00      1.00      1.00      3762



<구독 확률 예측 모델 해석>
- ROC AUC 0.512는 모델이 구독 여부를 예측하는 데 있어 거의 무작위 수준임을 의미
- 클래스 1(구독)의 낮은 precision(0.15)은 많은 오분류(즉, 실제 미구독인 고객을 구독으로 예측)가 존재한다는 것을 암시
- 반면, 클래스 1의 recall(0.83)은 대부분의 실제 구독 고객을 포착하는 경향이 있지만, 이는 오분류 수가 많아 F1-score가 낮게 나타남
- 전체적으로, 선택한 피처(구매 내역, 구매 주기, 배송지연 빈도 등)가 구독 여부를 잘 설명하지 못하거나, 데이터의 불균형(미구독: 3238, 구독: 524) 문제 때문에 모델이 제대로 학습되지 않은 것으로 파악됨

<자주 구매(구매주기 n개월 이하) 예측 모델 해석>
- ROC AUC 1.0와 모든 평가지표가 1.0에 가깝다는 것은 모델이 라벨을 완벽하게 분리하고 있다는 것을 의미
- 다만, support에서 알 수 있듯이 클래스 0(구매주기가 n개월 이상인 고객)은 46명에 불과하고, 대부분의 고객(3716명)이 이미 자주 구매(구매주기 n개월 이하)로 분류
- 이는 정의 상 n개월 이하(예: n=3개월, 약 90일 이하)라는 기준이 너무 낮거나, 대부분의 고객이 자주 구매하는 특성을 보이고 있음을 암시
- 따라서 모델의 완벽한 성능은 실제로 예측해야 할 문제가 간단하거나 라벨이 극도로 불균형하게 분포되어 있기 때문일 가능성 있다고 봄.

In [12]:
# 4. 다양한 입력변수를 실험하면서 모델 결과를 살펴보기
# ================================================================
# 예를 들어, 추가적인 피처(예: last_order_date로부터 경과 일수 등)를 추가할 수 있습니다.
# 아래는 고객의 마지막 주문일과 기준일 사이의 일수를 피처로 추가하는 예시입니다.

# 기준 날짜: df_sales의 최대 주문일
reference_date = df_sales['주문일시'].max()
customer_features['days_since_last_order'] = (reference_date - customer_features['last_order_date']).dt.days

# 추가 피처 목록
extended_feature_cols = feature_cols + ['days_since_last_order']

# 모델 1 확장: 구독 모델
X_sub_ext = customer_features[extended_feature_cols]
y_sub_ext = customer_features['구독여부']

X_train_sub_ext, X_test_sub_ext, y_train_sub_ext, y_test_sub_ext = train_test_split(X_sub_ext, y_sub_ext, test_size=0.3, random_state=42)

log_reg_sub_ext = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
log_reg_sub_ext.fit(X_train_sub_ext, y_train_sub_ext)

y_pred_proba_sub_ext = log_reg_sub_ext.predict_proba(X_test_sub_ext)[:, 1]
print("\n----- 확장 구독 모델 (추가 피처 포함) -----")
print("ROC AUC Score:", roc_auc_score(y_test_sub_ext, y_pred_proba_sub_ext))
print(classification_report(y_test_sub_ext, log_reg_sub_ext.predict(X_test_sub_ext)))

# 모델 2 확장: 자주 구매 모델
X_freq_ext = customer_features[extended_feature_cols]
y_freq_ext = customer_features['frequent_buyer']

X_train_freq_ext, X_test_freq_ext, y_train_freq_ext, y_test_freq_ext = train_test_split(X_freq_ext, y_freq_ext, test_size=0.3, random_state=42)

log_reg_freq_ext = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
log_reg_freq_ext.fit(X_train_freq_ext, y_train_freq_ext)

y_pred_proba_freq_ext = log_reg_freq_ext.predict_proba(X_test_freq_ext)[:, 1]
print("\n----- 확장 자주 구매 모델 (추가 피처 포함) -----")
print("ROC AUC Score:", roc_auc_score(y_test_freq_ext, y_pred_proba_freq_ext))
print(classification_report(y_test_freq_ext, log_reg_freq_ext.predict(X_test_freq_ext)))


----- 확장 구독 모델 (추가 피처 포함) -----
ROC AUC Score: 0.5150084398530805
              precision    recall  f1-score   support

           0       0.87      0.47      0.61      3238
           1       0.14      0.56      0.23       524

    accuracy                           0.48      3762
   macro avg       0.51      0.51      0.42      3762
weighted avg       0.77      0.48      0.55      3762


----- 확장 자주 구매 모델 (추가 피처 포함) -----
ROC AUC Score: 1.0
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       282
           1       1.00      1.00      1.00      3480

    accuracy                           1.00      3762
   macro avg       1.00      1.00      1.00      3762
weighted avg       1.00      1.00      1.00      3762



In [ ]:
<확장 구독 모델 해석>
구독 확률 예측 모델은 추가 피처(예: 최근 주문 경과 일수 등)를 포함했음에도 불구하고, 
여전히 구독 여부를 예측하는 데 있어서 거의 무작위 수준에 머무르고 있다.

- 제공된 피처들이 구독 여부를 설명하는 데 충분한 정보를 담고 있지 않거나,
- 데이터 불균형 또는 타깃 변수의 복잡성 때문에 모델이 학습하기 어려운 문제임을 시사

<확장 자주 구매 모델>
모델이 완벽한 성능을 보이고 있지만, 이는 데이터 분포의 심각한 불균형 때문일 가능성이 크다.

- 전체 고객 중 3716명이 자주 구매하는(구매주기 n개월 이하) 고객에 해당하고, 
    단 46명만이 자주 구매하지 않는 고객
- 이러한 극단적인 불균형은 모델이 거의 모든 고객을 자주 구매하는 클래스로 예측

-> n값을 1로 바꾸면 조금 나아지긴 하지만 여전히 상황은 같음. 위 모델 실행 결과는 n=1일 때 결과이고,
n값을 조정하시면 결과가 바로 바뀝니다.